In [ ]:
!pip install chromadb

In [ ]:
!pip install -U langchain-chroma

In [91]:
import os
import pysqlite3
import sys
sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")
import chromadb
from chromadb.config import Settings

In [92]:
import gradio as gr
from llama_index.llms.ollama import Ollama
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.llms.base import LLM

from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
#from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain.vectorstores import VectorStore
from typing import List, Any
from chromadb import Client
from langchain.chains import LLMChain, ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain

In [93]:
chroma_client = chromadb.Client()

In [94]:
# switch `create_collection` to `get_or_create_collection` to avoid creating a new collection every time
collection_name="dwarf_collectors"
collection = chroma_client.get_or_create_collection(name=collection_name)

In [95]:
# Define a custom LangChain wrapper for the Ollama LLaMA model
class OllamaLangChain(LLM):
    model_name: str = "llama3.2:latest"  # Declared field
    request_timeout: int = 90           # Declared field

    def __init__(self, model_name="llama3.2:latest", request_timeout=30):
        super().__init__()
        self.model_name = model_name
        self.request_timeout = request_timeout
        # Use object.__setattr__ to bypass Pydantic's field validation
        object.__setattr__(self, "_client", Ollama(model=model_name, request_timeout=request_timeout))
    
    @property
    def _llm_type(self):
        return "ollama"

    def _call(self, prompt, stop=None):
        # Access the private client
        client = object.__getattribute__(self, "_client")
        response = client.complete(prompt)
        return response.text

In [96]:
# Read the text file
with open("docs/dwarf.txt", "r", encoding="utf-8") as file:
    text = file.read()

# Create a list of a single document
documents = [Document(page_content=text)]

In [97]:
embeddings = OllamaEmbeddings(
    model="llama3.2"
)

In [98]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 256,
    chunk_overlap  = 40
    
)
# Split the text into chunks
splitted_txt = text_splitter.split_documents(documents)

In [99]:
# Output the result
for chunk in splitted_txt:
    print(chunk.page_content)
    print("\n")

Dwarf
Dwarf Traits
Your dwarf character has an assortment of inborn
abilities, part and parcel of dwarven nature.
Ability Score Increase. Your Constitution score
increases by 2.
Age. Dwarves mature at the same rate as humans,


but they’re considered young until they reach the
age of 50. On average, they live about 350 years.
Alignment. Most dwarves are lawful, believing
firmly in the benefits of a well-­‐‑ordered society. They


tend toward good as well, with a strong sense of fair
play and a belief that everyone deserves to share in
the benefits of a just order.
Size. Dwarves stand between 4 and 5 feet tall and
average about 150 pounds. Your size is Medium.


Speed. Your base walking speed is 25 feet. Your
speed is not reduced by wearing heavy armor.
Darkvision. Accustomed to life underground, you
have superior vision in dark and dim conditions. You
can see in dim light within 60 feet of you as if it were


bright light, and in darkness as if it were dim light.
You can’t discern colo

In [100]:
vectorstore = Chroma(
    embedding_function=embeddings,
    collection_name=collection_name,
    client=chroma_client
)


In [101]:
# Add documents to the collection
vectorstore.add_documents(splitted_txt)

['57ea77e7-bdc0-4c95-9892-63f952d6d44a',
 'a4002c0d-d669-4b4d-ab85-a7aa3ee42988',
 '59e95f09-68bb-4f88-9814-9daf14556863',
 '77d336c4-2b7a-4ae0-a935-292b17bb08fe',
 '4abd1d85-b29d-4ee2-95ac-b94dd1340fdd',
 '7eebb113-0fd5-4105-969c-b0c237fbabd6',
 '05da72dd-1a0e-42cb-8e0e-0ba5b6fc10ee',
 '22ccdbdc-0ed6-414f-9c11-20b270f93590',
 'c017197a-3fa9-4808-8b8e-a545c75076ad',
 'f878b2f6-5e55-4a23-8b30-d9f576e1d718',
 '9d39c731-0f30-4483-9071-16391b08daa0']

In [102]:
retriever = vectorstore.as_retriever()

In [103]:
llm = OllamaLangChain()
memory = ConversationBufferMemory()  # Store chat history


In [128]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# See full prompt at https://smith.langchain.com/hub/rlm/rag-prompt
prompt = hub.pull("rlm/rag-prompt")

# Define your custom prompt template
custom_prompt_template = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. Keep the answer concise, with a maximum of three sentences.

Question: {question}

Context: {context}

Answer:
"""

# Create a PromptTemplate using your custom template
custom_prompt = PromptTemplate(template=custom_prompt_template, input_variables=["question", "context"])


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def get_context_for_question(question):
    # Create a VectorStoreRetriever using 'as_retriever()' on your vectorstore
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})  # Adjust as needed
    
    # Retrieve the relevant documents based on the input question
    context = retriever.get_relevant_documents(question)
    
    # Format the retrieved context into a string
    formatted_context = format_docs(context)
    return formatted_context

qa_chain = (
    {
        #"context": RunnablePassthrough(func=get_context_for_question),
        "context": vectorstore.as_retriever() | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
    
)

qa_chain.invoke("What about language in Dwarf")

/home/capmars/SemAgent/.venv/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


'As a Dwarf, you can speak, read, and write Common and Dwarvish. However, speaking another language may not be as proficient due to the unique characteristics of Dwarvish, which often influence other languages spoken by Dwarves. This can result in a lower proficiency bonus when trying to communicate in a foreign tongue.'

In [124]:
# Define the function to handle the Gradio interface input/output
def answer_question(question):
    # Use the qa_chain to answer the question
    return qa_chain.invoke(question)

In [125]:
# Gradio interface
interface = gr.Interface(
    fn=answer_question,
    inputs="text",
    outputs="text",
    title="LLaMA 3 Chatbot with LangChain",
    description="Chat with a LLaMA 3-based model via Ollama and LangChain!"
)
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7871
Running on public URL: https://9ec9bc2aefb0b2fc0c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
